# Install dan Download Category Encoder


In [ ]:
pip install category_encoders

     |████████████████████████████████| 81kB 3.3MB/s 


# Install Ipython Autotime

In [ ]:
!pip install ipython-autotime
%load_ext autotime

time: 208 µs (started: 2021-01-09 15:00:25 +00:00)


# Import Libraries

In [ ]:
import keras
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from matplotlib import pyplot
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn import svm
from sklearn.model_selection import KFold
from category_encoders.target_encoder import TargetEncoder

from sklearn import tree
import matplotlib.pyplot as plt

from category_encoders.target_encoder import TargetEncoder

time: 14.5 ms (started: 2021-01-09 15:08:55 +00:00)


# Load Data

Read data dan drop kolom yang tidak dibutuhkan

In [ ]:
df = pd.read_csv("winemag-data_first150k.csv")
df.drop(columns=['designation', 'price', 'region_1', 'variety', 'winery', 'description', 'Unnamed: 0', 'region_2'], inplace=True)
df

,country,points,province
0,US,96,California
1,Spain,96,Northern Spain
2,US,96,California
3,US,96,Oregon
4,France,95,Provence
...,...,...,...
150925,Italy,91,Southern Italy
150926,France,91,Champagne
150927,Italy,91,Southern Italy
150928,France,90,Champagne


time: 818 ms (started: 2021-01-09 15:08:57 +00:00)


Assign df_disc dengan data yang tidak memiliki nilai null

In [ ]:
df_disc = df.copy()
for col in df_disc.columns:
    df_disc = df_disc[df_disc[col].notnull()]

df_disc

,country,points,province
0,US,96,California
1,Spain,96,Northern Spain
2,US,96,California
3,US,96,Oregon
4,France,95,Provence
...,...,...,...
150925,Italy,91,Southern Italy
150926,France,91,Champagne
150927,Italy,91,Southern Italy
150928,France,90,Champagne


time: 56.3 ms (started: 2021-01-09 15:09:00 +00:00)


Assign df_disc dengan data yang pada kolom "winery" memiliki nilai yang duplicate

In [ ]:
df_disc = df_disc[df_disc.duplicated(subset=["country"], keep=False)]
print(df_disc)

       country  points            province
0           US      96          California
1        Spain      96      Northern Spain
2           US      96          California
3           US      96              Oregon
4       France      95            Provence
...        ...     ...                 ...
150925   Italy      91      Southern Italy
150926  France      91           Champagne
150927   Italy      91      Southern Italy
150928  France      90           Champagne
150929   Italy      90  Northeastern Italy

[150924 rows x 3 columns]
time: 42.7 ms (started: 2021-01-09 15:09:03 +00:00)


Assign df_disc dengan data yang pada kolom "designation" memiliki nilai yang duplicate

In [ ]:
df_disc = df_disc[df_disc.duplicated(subset=["province"], keep=False)]
print(df_disc)

       country  points            province
0           US      96          California
1        Spain      96      Northern Spain
2           US      96          California
3           US      96              Oregon
4       France      95            Provence
...        ...     ...                 ...
150925   Italy      91      Southern Italy
150926  France      91           Champagne
150927   Italy      91      Southern Italy
150928  France      90           Champagne
150929   Italy      90  Northeastern Italy

[150882 rows x 3 columns]
time: 39 ms (started: 2021-01-09 15:09:06 +00:00)


Mendiskritkan target menjadi kontinu menggunakan qcut dari pandas dengan parameter pd.qcut(kolom, jumlah_kelas, label) dan replace kolom target (price) dengan angka

In [ ]:
df_disc['points'] = pd.qcut(df_disc['points'], 2, labels=['low', 'high'])
df_disc['points'].replace({'low': 0, 'high': 1}, inplace=True)
df_disc

,country,points,province
0,US,1,California
1,Spain,1,Northern Spain
2,US,1,California
3,US,1,Oregon
4,France,1,Provence
...,...,...,...
150925,Italy,1,Southern Italy
150926,France,1,Champagne
150927,Italy,1,Southern Italy
150928,France,1,Champagne


time: 141 ms (started: 2021-01-09 15:09:07 +00:00)


# Encoding

Menggunakan Target Encoder yang meng-encode berdasarkan target. Value yang diberikan antara 0 sampai 1

In [ ]:
te = TargetEncoder()

y = df_disc['points'].values
X = df_disc.drop(columns=['points']).values
te.fit(X, y)
X = te.transform(X)

X.head()

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,0,1
0,0.401548,0.386515
1,0.259918,0.274734
2,0.401548,0.386515
3,0.401548,0.483766
4,0.532894,0.475024


time: 234 ms (started: 2021-01-09 15:09:11 +00:00)


# Training

## ANN

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=1)

time: 19.9 ms (started: 2021-01-09 03:28:31 +00:00)


In [ ]:
model=keras.models.Sequential([
    keras.layers.Dense(512, input_dim = X_train.shape[1], activation='relu'),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=128, activation='relu'),

    keras.layers.Dense(units=1, activation="sigmoid"),
],name="Initial_model",)
model.summary()

Model: "Initial_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               1536      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 165,889
Trainable params: 165,889
Non-trainable params: 0
_________________________________________________________________
time: 289 ms (started: 2021-01-09 03:28:31 +00:00)


In [ ]:
learning_rate = 0.005
optimizer = keras.optimizers.Adam(lr=learning_rate)

time: 1.63 ms (started: 2021-01-09 03:28:31 +00:00)


In [ ]:
model.compile(optimizer=optimizer,
            loss='binary_crossentropy', metrics=['accuracy'])

time: 25.4 ms (started: 2021-01-09 03:28:31 +00:00)


In [ ]:
early_stopping_monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=1000,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True
)

time: 2.73 ms (started: 2021-01-09 03:28:31 +00:00)


In [ ]:
history = model.fit(X_train, y_train,
                    epochs=500, batch_size=1024,
                    validation_data=(X_test, y_test), 
                    verbose=1, callbacks=[early_stopping_monitor])

Epoch 1/500
45/45 [==============================] - 3s 47ms/step - loss: 0.6544 - accuracy: 0.6194 - val_loss: 0.6283 - val_accuracy: 0.6423
Epoch 2/500
45/45 [==============================] - 2s 42ms/step - loss: 0.6291 - accuracy: 0.6401 - val_loss: 0.6273 - val_accuracy: 0.6417
Epoch 3/500
45/45 [==============================] - 2s 42ms/step - loss: 0.6321 - accuracy: 0.6360 - val_loss: 0.6270 - val_accuracy: 0.6417
Epoch 4/500
45/45 [==============================] - 2s 45ms/step - loss: 0.6309 - accuracy: 0.6338 - val_loss: 0.6272 - val_accuracy: 0.6428
Epoch 5/500
45/45 [==============================] - 2s 43ms/step - loss: 0.6288 - accuracy: 0.6405 - val_loss: 0.6284 - val_accuracy: 0.6422
Epoch 6/500
45/45 [==============================] - 2s 43ms/step - loss: 0.6294 - accuracy: 0.6404 - val_loss: 0.6282 - val_accuracy: 0.6412
Epoch 7/500
45/45 [==============================] - 2s 42ms/step - loss: 0.6278 - accuracy: 0.6380 - val_loss: 0.6268 - val_accuracy: 0.6428
Epoch 

In [ ]:
_, train_acc = model.evaluate(X_train, y_train, verbose=0)
_, test_acc = model.evaluate(X_test, y_test, verbose=0)

time: 4.81 s (started: 2021-01-09 03:45:34 +00:00)


In [ ]:
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

Train: 0.640, Test: 0.642
time: 1.68 ms (started: 2021-01-09 03:45:39 +00:00)


## Decision Tree

In [ ]:
param_dt = {
    'test_size': [0.9, 0.7, 0.5, 0.3, 0.1],
    'max_depth': [3, 5, 7],
    'criterion': ['gini', 'entropy']
}

time: 1.95 ms (started: 2021-01-09 03:45:39 +00:00)


In [ ]:
row_format ="{:>10}" * (len(param_dt) + 2)
print(row_format.format('t_size', 'max_depth', 'criterion', 'akurasi', 'f1'))
acc_bfr = -math.inf
for i in param_dt['test_size']:
  for j in param_dt['max_depth']:
    for k in param_dt['criterion']:
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=42)
      dt = DecisionTreeClassifier(random_state=42, max_depth=j, criterion=k)
      dt.fit(X_train, y_train)

      # print(y_test)
      pred = dt.predict(X_test)
      acc = metrics.accuracy_score(y_test, pred)
      f1 = metrics.f1_score(y_test, pred)
      if(acc > acc_bfr):
        acc_bfr = acc
        best_param_dt = {'t_size': i, 'max_depth': j, 'criterion': k, 'accuracy': acc, 'f1': f1}
        best_dt = dt
      print(row_format.format(str(i), str(j), str(k), '%.3f' % acc, '%.3f' % f1))
print('best param: ', best_param_dt)     

    t_size max_depth criterion   akurasi        f1
       0.9         3      gini     0.642     0.488
       0.9         3   entropy     0.642     0.488
       0.9         5      gini     0.640     0.491
       0.9         5   entropy     0.640     0.491
       0.9         7      gini     0.639     0.500
       0.9         7   entropy     0.640     0.501
       0.7         3      gini     0.639     0.527
       0.7         3   entropy     0.639     0.527
       0.7         5      gini     0.642     0.488
       0.7         5   entropy     0.642     0.488
       0.7         7      gini     0.641     0.494
       0.7         7   entropy     0.641     0.494
       0.5         3      gini     0.639     0.526
       0.5         3   entropy     0.639     0.526
       0.5         5      gini     0.642     0.491
       0.5         5   entropy     0.642     0.492
       0.5         7      gini     0.642     0.491
       0.5         7   entropy     0.642     0.491
       0.3         3      gini 

## SVM

In [ ]:
param_svm = {
  'test_size': [0.9, 0.7, 0.5, 0.3, 0.1],  
  'C'    : [1, 10, 100, 0.1],
  'gamma': [0.1, 0.01, 0.001, 1.0],
  'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
}

time: 3.5 ms (started: 2021-01-09 03:45:41 +00:00)


In [ ]:
row_format ="{:>10}" * (len(param_svm) + 2)
print(row_format.format(*param_svm, 'akurasi', 'f1'))
acc_bfr = -math.inf
for i in param_svm['test_size']:
  for j in param_svm['C']:
    for k in param_svm['gamma']:
      for l in param_svm['kernel']:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=42)
        svc = SVC(C=j, gamma=k, kernel=l)
        svc.fit(X_train, y_train)

        # print(y_test)
        pred = svc.predict(X_test)
        acc = metrics.accuracy_score(y_test, pred)
        f1 = metrics.f1_score(y_test, pred)
        if(acc > acc_bfr):
          acc_bfr = acc
          best_param_svm = {'t_size': i, 'C': j, 'gamma': k, 'kernel': l, 'accuracy': acc, 'f1': f1}
        print(row_format.format(str(i), str(j), str(k), str(l), '%.3f' % acc, '%.3f' % f1))
print('best param: ', best_param_svm)     

 test_size         C     gamma    kernel   akurasi        f1
       0.9         1       0.1    linear     0.641     0.515
       0.9         1       0.1      poly     0.595     0.000
       0.9         1       0.1       rbf     0.642     0.472
       0.9         1       0.1   sigmoid     0.642     0.473
       0.9         1      0.01    linear     0.641     0.515
       0.9         1      0.01      poly     0.595     0.000
       0.9         1      0.01       rbf     0.598     0.016
       0.9         1      0.01   sigmoid     0.595     0.000
       0.9         1     0.001    linear     0.641     0.515
       0.9         1     0.001      poly     0.595     0.000
       0.9         1     0.001       rbf     0.595     0.000
       0.9         1     0.001   sigmoid     0.595     0.000
       0.9         1       1.0    linear     0.641     0.515
       0.9         1       1.0      poly     0.642     0.491
       0.9         1       1.0       rbf     0.642     0.475
       0.9         1    

## KNN

In [ ]:
param_knn = {
  'test_size': [0.9, 0.7, 0.5, 0.3, 0.1],  
  'n_neighbor': [3, 5, 7],
  'weights': ['uniform', 'distance'],
  'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
}

time: 1.47 ms (started: 2021-01-09 15:09:18 +00:00)


In [ ]:
row_format ="{:>10}" * (len(param_knn) + 2)
print(row_format.format(*param_knn, 'akurasi', 'f1'))
acc_bfr = -math.inf
for i in param_knn['test_size']:
  for j in param_knn['n_neighbor']:
    for k in param_knn['weights']:
      for l in param_knn['algorithm']:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=42)
        knn = KNeighborsClassifier(n_neighbors=j, weights=k, algorithm=l)
        knn.fit(X_train, y_train)

        # print(y_test)
        pred = knn.predict(X_test)
        acc = metrics.accuracy_score(y_test, pred)
        f1 = metrics.f1_score(y_test, pred)
        if(acc > acc_bfr):
          acc_bfr = acc
          best_param_knn = {'t_size': i, 'n_beighbors': j, 'weights': k, 'algorithm': l, 'accuracy': acc, 'f1': f1}
        print(row_format.format(str(i), str(j), str(k), str(l), '%.3f' % acc, '%.3f' % f1))
print('best param: ', best_param_knn)     

 test_sizen_neighbor   weights algorithm   akurasi        f1
       0.9         3   uniform      auto     0.597     0.378
       0.9         3   uniform ball_tree     0.596     0.378
       0.9         3   uniform   kd_tree     0.597     0.378
       0.9         3   uniform     brute     0.528     0.477
       0.9         3  distance      auto     0.597     0.377
       0.9         3  distance ball_tree     0.597     0.377
       0.9         3  distance   kd_tree     0.597     0.377
       0.9         3  distance     brute     0.529     0.476
       0.9         5   uniform      auto     0.604     0.367
       0.9         5   uniform ball_tree     0.604     0.367
       0.9         5   uniform   kd_tree     0.604     0.367
       0.9         5   uniform     brute     0.553     0.547
       0.9         5  distance      auto     0.603     0.366
       0.9         5  distance ball_tree     0.603     0.365
       0.9         5  distance   kd_tree     0.603     0.366
       0.9         5  di